# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

<Client: 'inproc://10.1.0.4/7699/1' processes=1 threads=4, memory=2.00 GB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1012,Ursula,-0.6751199140158914,0.5242863232669723
2000-01-01 00:00:01,1013,Michael,-0.8287582596363172,0.3485932416088935
2000-01-01 00:00:02,989,Oliver,-0.06328199081173214,0.1567995020302515
2000-01-01 00:00:03,1018,Michael,0.07949954234818102,0.2977398576529544
2000-01-01 00:00:04,1031,Xavier,-0.7436442623365327,0.4902924446313284
2000-01-01 00:00:05,968,George,0.5852857175014177,0.26433641857474166
2000-01-01 00:00:06,1008,Yvonne,-0.7862519084925619,0.12885065259999218
2000-01-01 00:00:07,977,Edith,0.7121215209189704,0.9457770026013943
2000-01-01 00:00:08,981,Tim,-0.8739022379360417,0.4778121707489018


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1036,Ursula,0.17987944529618605,-0.8432806741360768
2000-01-30 00:00:01,1030,Jerry,0.03989682365094249,0.9356467722819963
2000-01-30 00:00:02,992,Oliver,-0.23855915084268609,0.8585771522478809
2000-01-30 00:00:03,1008,George,0.22034275205792309,0.5898649303410246
2000-01-30 00:00:04,1071,Patricia,-0.8978474070878939,0.7825250578936171
2000-01-30 00:00:05,983,Patricia,0.8547066699959027,0.33646788309230136
2000-01-30 00:00:06,984,Kevin,-0.7556868143774964,0.4483357649286954
2000-01-30 00:00:07,1007,Patricia,-0.5193856035879687,-0.5387739784661199
2000-01-30 00:00:08,1021,Ray,0.8157069195450624,0.5630515056787551


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

timestamp    id     name         x         y
0  2000-01-01 00:00:00  1012   Ursula -0.675120  0.524286
1  2000-01-01 00:00:01  1013  Michael -0.828758  0.348593
2  2000-01-01 00:00:02   989   Oliver -0.063282  0.156800
3  2000-01-01 00:00:03  1018  Michael  0.079500  0.297740
4  2000-01-01 00:00:04  1031   Xavier -0.743644  0.490292

In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
               timestamp     id    name        x        y
npartitions=30                                           
                  object  int64  object  float64  float64
                     ...    ...     ...      ...      ...
...                  ...    ...     ...      ...      ...
                     ...    ...     ...      ...      ...
                     ...    ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [10]:
df.head()

timestamp    id     name         x         y
0  2000-01-01 00:00:00  1012   Ursula -0.675120  0.524286
1  2000-01-01 00:00:01  1013  Michael -0.828758  0.348593
2  2000-01-01 00:00:02   989   Oliver -0.063282  0.156800
3  2000-01-01 00:00:03  1018  Michael  0.079500  0.297740
4  2000-01-01 00:00:04  1031   Xavier -0.743644  0.490292

## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

Dask DataFrame Structure:
                     timestamp     id    name        x        y
npartitions=30                                                 
                datetime64[ns]  int64  object  float64  float64
                           ...    ...     ...      ...      ...
...                        ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.1 s, sys: 558 ms, total: 4.66 s
Wall time: 4.04 s


name
Alice       0.000261
Bob        -0.001519
Charlie    -0.000102
Dan         0.000006
Edith       0.000590
Frank      -0.002222
George      0.002666
Hannah     -0.001835
Ingrid      0.000479
Jerry       0.000113
Kevin      -0.000711
Laura      -0.002990
Michael    -0.000744
Norbert     0.003016
Oliver     -0.000469
Patricia    0.000794
Quinn       0.000784
Ray         0.000091
Sarah       0.000447
Tim        -0.000512
Ursula      0.000900
Victor      0.004620
Wendy      -0.001705
Xavier      0.000969
Yvonne     -0.000286
Zelda       0.003652
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

Dask DataFrame Structure:
                     timestamp     id    name        x        y
npartitions=30                                                 
                datetime64[ns]  int64  object  float64  float64
                           ...    ...     ...      ...      ...
...                        ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
Dask Name: read-parquet, 30 tasks

In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.3 s, sys: 107 ms, total: 1.41 s
Wall time: 1.31 s


name
Alice       0.000261
Bob        -0.001519
Charlie    -0.000102
Dan         0.000006
Edith       0.000590
Frank      -0.002222
George      0.002666
Hannah     -0.001835
Ingrid      0.000479
Jerry       0.000113
Kevin      -0.000711
Laura      -0.002990
Michael    -0.000744
Norbert     0.003016
Oliver     -0.000469
Patricia    0.000794
Quinn       0.000784
Ray         0.000091
Sarah       0.000447
Tim        -0.000512
Ursula      0.000900
Victor      0.004620
Wendy      -0.001705
Xavier      0.000969
Yvonne     -0.000286
Zelda       0.003652
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.31 s, sys: 59.5 ms, total: 1.37 s
Wall time: 1.29 s


name
Alice       0.000261
Bob        -0.001519
Charlie    -0.000102
Dan         0.000006
Edith       0.000590
Frank      -0.002222
George      0.002666
Hannah     -0.001835
Ingrid      0.000479
Jerry       0.000113
Kevin      -0.000711
Laura      -0.002990
Michael    -0.000744
Norbert     0.003016
Oliver     -0.000469
Patricia    0.000794
Quinn       0.000784
Ray         0.000091
Sarah       0.000447
Tim        -0.000512
Ursula      0.000900
Victor      0.004620
Wendy      -0.001705
Xavier      0.000969
Yvonne     -0.000286
Zelda       0.003652
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html